# AE413: Optimization Techniques in Engineering
## Assignment 1: Formulation of optimization problems and solution using Python
---
**Name**: Gaurav Gupta; SC21B026; B.Tech Aerospace Engineering

In [142]:
import numpy as np
from scipy.optimize import minimize, linprog, milp, Bounds, LinearConstraint

### Question 1

In [143]:
h = 50
v = 90
theta0 = [45*np.pi/180]
bound = [(0, 80*np.pi/180)]
def D(O):
    global h, v
    g = 9.81
    return -(v*v*np.sin(2*O)/g/2 + np.sqrt((v*np.sin(O)/g)**2 + 2*h/g)*v*np.cos(O))

result = minimize(D, theta0, method='SLSQP', bounds=bound)
print(f"Optimized value of D is {-result.fun:.3f} m for a value of theta = {result.x[0] * 180 / np.pi:.3f} degrees.")

Optimized value of D is 874.259 m for a value of theta = 43.363 degrees.


### Question 2

In [144]:
i = 3
j = 2
k = 5
Qij = [10, 7, 15, 20, 15, 8]
Ckj = [100, 200, 250, 150, 400, 450, 300, 150, 250, 300]
Aki = np.zeros((1, k * i))

bounds = Bounds(0, 1)  # 0 <= x_i <= 1
integrality = np.ones(15)  # 15 integer variables

# Coefficient Matrix
c = (np.reshape(Ckj, (k, j)) @ np.reshape(Qij, (j, i)))
c = c.flatten()

# Constraints
# A1: Ensuring each k row has a sum of 1
A1 = np.zeros((k, i * k))
for n in range(k):
    A1[n, n * i:(n + 1) * i] = 1

B1_u = np.ones(k)
B1_l = -np.inf * np.ones(k)

# A2: Sum constraints for i elements across different k
A2 = np.zeros((i, i * k))
for n in range(i):
    A2[n, n::i] = 1

B2_u = np.ones(i)
B2_l = np.copy(B2_u)

# Combining constraints
A = np.vstack([A1, A2])
b_u = np.hstack([B1_u, B2_u])
b_l = np.hstack([B1_l, B2_l])

cons = LinearConstraint(A, b_l, b_u)

# Minimize c @ x
res = milp(c=c, integrality=integrality, bounds=bounds, constraints=cons)

print(f"Minimum cost of transport is Rs. {res.fun:.2f}.")
print(res.x.reshape((5,3)))

Minimum cost of transport is Rs. 12950.00.
[[-0.  0.  1.]
 [ 1. -0.  0.]
 [ 0.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]


### Question 3

In [145]:
# Given data
ri = np.array([1100, 1000, 900])
Sij = np.array([[3000, 3500], [2000, 2500], [6000, 4000]])
Gi = np.array([200, 310, 420])
Cj = np.array([460, 560])
Lj = np.array([26000, 21000])

# Number of suppliers and plants
num_suppliers = 3
num_plants = 2

# Objective function coefficients (profit per unit)
selling_price = 50000
c = -(
    selling_price -  # Revenue
    np.repeat(ri, num_plants) -  # Supplier cost
    Sij.flatten() -  # Shipping cost
    np.tile(Lj, num_suppliers)  # Labor cost
)

# Inequality constraints (Ax <= b)
A_ub = []

# Plant capacity constraints: Sum of supplies to each plant <= Plant capacity
for j in range(num_plants):
    constraint = np.zeros(num_suppliers * num_plants)
    for i in range(num_suppliers):
        constraint[i * num_plants + j] = 1
    A_ub.append(constraint)

b_ub = Cj.tolist()  # Plant capacities

# Supplier capacity constraints: Sum of supplies from each supplier <= Supplier capacity
for i in range(num_suppliers):
    constraint = np.zeros(num_suppliers * num_plants)
    for j in range(num_plants):
        constraint[i * num_plants + j] = 1
    A_ub.append(constraint)

b_ub.extend(Gi.tolist())  # Supplier capacities

# Convert to numpy arrays
A_ub = np.array(A_ub)
b_ub = np.array(b_ub)

# Bounds for each variable (non-negative quantities)
bounds = [(0, None)] * (num_suppliers * num_plants)

# Initial guess (not used in linprog, but defining for clarity)
initial_guess = np.zeros(6)

# Perform linear programming optimization (linprog minimizes, so we negate c for maximization)
result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

# Reshape the result to a 2D array
Q_opt = result.x.reshape(num_suppliers, num_plants)

# Print the results
max_profit = -result.fun  # Negate because we minimized the negative profit
print(f"Maximum profit is Rs. {max_profit:.3f}")
print(f"Plant A purchased {Q_opt[0,0]:.3f} Tonnes, {Q_opt[1,0]:.3f} Tonnes, and {Q_opt[2,0]:.3f} Tonnes from Supplier 1, 2, and 3, respectively.")
print(f"Plant B purchased {Q_opt[0,1]:.3f} Tonnes, {Q_opt[1,1]:.3f} Tonnes, and {Q_opt[2,1]:.3f} Tonnes from Supplier 1, 2, and 3, respectively.")

Maximum profit is Rs. 21242000.000
Plant A purchased 200.000 Tonnes, 170.000 Tonnes, and 0.000 Tonnes from Supplier 1, 2, and 3, respectively.
Plant B purchased 0.000 Tonnes, 140.000 Tonnes, and 420.000 Tonnes from Supplier 1, 2, and 3, respectively.


### Question 4

In [146]:
import numpy as np
from scipy.optimize import linprog

i = 2
j = 2
Mj = [20, 18]
Tij = [3, 4, 3, 2]  # Time constraints
Pi = [200, 300]     # Profit coefficients for i
P3 = 150            # Profit coefficient for P3
P4 = 75             # Profit coefficient for P4
n = 5               # By-product multiplier
K = 1               # Upper bound for x3

# Coefficient matrix for objective function
c = -np.array(Pi + [P3, -P4])

# Boundaries for the decision variables (x1, x2 >= 0, 0 <= x3 <= 1, x4 >= 0)
bounds = [(0, None), (0, None), (0, K), (0, None)]  # Adding bound for x3

# Inequality constraint (time constraint): Tij @ x <= Mj
Au = np.array([[Tij[0], Tij[2], 0, 0],
               [Tij[1], Tij[3], 0, 0]])

Bu = np.array(Mj)

# Equality constraint (by-product constraint): x2 - n * (x3 + x4) = 0
Aeq = np.array([[0, 1, -n, -n]])
Beq = np.array([0])

# Solving the optimization problem
res = linprog(c, A_ub=Au, b_ub=Bu, A_eq=Aeq, b_eq=Beq, bounds=bounds)

# Display the result
print(res)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -2125.0
              x: [ 0.000e+00  6.667e+00  1.000e+00  3.333e-01]
            nit: 1
          lower:  residual: [ 0.000e+00  6.667e+00  1.000e+00  3.333e-01]
                 marginals: [ 8.500e+01  0.000e+00  0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf  0.000e+00        inf]
                 marginals: [ 0.000e+00  0.000e+00 -2.250e+02  0.000e+00]
          eqlin:  residual: [ 0.000e+00]
                 marginals: [-1.500e+01]
        ineqlin:  residual: [ 0.000e+00  4.667e+00]
                 marginals: [-9.500e+01 -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0


### Question 5

In [147]:
i = 2
j = 5
Vj = np.array([0.1, 0.02, 0.3, 0.057, 0.04])  # Volumes
Wj = np.array([10, 2.7, 36, 6.9, 0.5])        # Weights
V = 1                                         # Total volume constraint
W = 75                                        # Total weight constraint

bounds = Bounds(0, 1)  # 0 <= x_i <= 1 (continuous)

integrality = np.ones(i * j)  # 10 integer variables (2i * j)

# Coefficient matrix for objective function
c = np.column_stack((Vj, Vj))  # Coefficients for i1 and i2 variables
c = -c.flatten()               # Minimize the negative of the objective

# Volume constraints: sum of volumes for i <= V
A1 = np.zeros((i, i * j))
for n in range(i):
    idx = np.arange(j) * 2 + n  # Selecting the correct variables for each i
    A1[n, idx] = Vj

B1u = [V, V]  # Upper bound for volume constraints
B1l = [0, 0]  # Lower bound for volume constraints

# Weight constraints: sum of weights for i <= W
A2 = np.zeros((i, i * j))
for n in range(i):
    idx = np.arange(j) * 2 + n  # Selecting the correct variables for each i
    A2[n, idx] = Wj

B2u = [W, W]  # Upper bound for weight constraints
B2l = [0, 0]  # Lower bound for weight constraints

# Selection constraints: sum across i for each j <= 1
A3 = np.zeros((j, i * j))
for n in range(j):
    A3[n, [n * 2, n * 2 + 1]] = 1  # Sum of x1 and x2 for each j <= 1

B3u = [1] * j  # Upper bound: At most 1 selection per j
B3l = [0] * j  # Lower bound: Non-negative

# Combining constraints
A = np.vstack([A1, A2, A3])
b_u = np.hstack([B1u, B2u, B3u])
b_l = np.hstack([B1l, B2l, B3l])

# Define the constraints using the LinearConstraint object
cons = LinearConstraint(A, b_l, b_u)

# Solving the MILP problem
res = milp(c=c, integrality=integrality, bounds=bounds, constraints=cons)

# Display the result
print(res)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -0.517
              x: [ 1.000e+00  0.000e+00  1.000e+00  0.000e+00  1.000e+00
                   0.000e+00  1.000e+00  0.000e+00  1.000e+00  0.000e+00]
 mip_node_count: 0
 mip_dual_bound: -0.517
        mip_gap: 0.0


### Question 6

In [148]:
i = 3  # Number of decision variables
ai = [0.5, 1, 0.75]  # ai values for the problem
bi = [3, 2, 1.5]     # bi values for the problem
d = 500              # Constant value
epsilon = 10       # Small tolerance for strict inequality

# Objective function: Maximize the time based on x
def OF(x):
    t = np.zeros(i)
    t[0] = x[0] / bi[0] + (d - x[0]) / ai[0]
    for n in [1, 2]:
        t[n] = x[n - 1] / ai[n] + (x[n] - x[n - 1]) / bi[n] + (d - x[n]) / ai[n]
    return np.max(t)

# Constraints setup with small epsilon to enforce strict inequality
constraints = [
    {'type': 'ineq', 'fun': lambda x: x[0]- epsilon},                      # x[0] > 0
    {'type': 'ineq', 'fun': lambda x: x[1] - x[0] - epsilon},     # x[1] > x[0]
    {'type': 'ineq', 'fun': lambda x: x[2] - x[1] - epsilon},     # x[2] > x[1]
    {'type': 'ineq', 'fun': lambda x: d - x[2]},                  # d >= x[2]
    {'type': 'ineq', 'fun': lambda x: x[0] / ai[1] - x[0] / bi[0]},  # Additional constraint involving x[0]
    {'type': 'ineq', 'fun': lambda x: x[1] / ai[2] - (x[1] - x[0]) / bi[1]}  # Additional constraint involving x[1] and x[0]
]

# Bounds for the variables (0 <= x <= d for each variable)
bound = [(0, d)] * i

# Initial guess (starting point)
ig = [0] * i

# Solve the optimization problem using SLSQP
res = minimize(OF, ig, method='SLSQP', bounds=bound, constraints=constraints)

# Output the results
print(res)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 528.5714293221455
       x: [ 2.829e+02  2.929e+02  5.000e+02]
     nit: 26
     jac: [ 0.000e+00  6.667e-01 -6.667e-01]
    nfev: 135
    njev: 26


### Question 7

In [149]:
# Given Data
Mj = np.array([8000, 10000, 9000])  # Maximum availability of each coffee variety
Cj = np.array([120, 130, 100])  # Cost of each coffee variety
Si = np.array([300, 320, 280])  # Selling price for each blend

# Objective function to maximize profit
def profit(x):
    Qij = x.reshape(3, 3)  # Reshape x to a 2D array (3 Blends x 3 Coffee Varieties)
    sellingPrice = np.dot(np.sum(Qij, axis=1), Si)
    costPrice = np.dot(np.sum(Qij, axis=0), Cj)
    return -(sellingPrice - costPrice)

# Define constraints
constraints = [
    {'type': 'ineq', 'fun': lambda x: Mj - np.sum(x.reshape(3, 3), axis=0)},  # Availability constraints
    {'type': 'ineq', 'fun': lambda x: 25000 - np.sum(x)},  # Total capacity constraint
    {'type': 'ineq', 'fun': lambda x: np.sum(x.reshape(3, 3)[0, :]) - 5000},  # Blend 1 minimum amount constraint
    {'type': 'ineq', 'fun': lambda x: 0.2 * np.sum(x.reshape(3, 3)[0, :]) - x.reshape(3, 3)[0, 0]},  # Blend 1 coffee variety 1 proportion constraint
    {'type': 'ineq', 'fun': lambda x: x.reshape(3, 3)[0, 0] - 0.1 * np.sum(x.reshape(3, 3)[0, :])},  # Blend 1 coffee variety 1 lower proportion constraint
    {'type': 'ineq', 'fun': lambda x: 0.35 * np.sum(x.reshape(3, 3)[2, :]) - x.reshape(3, 3)[2, 1]},  # Blend 3 coffee variety 2 proportion constraint
    {'type': 'ineq', 'fun': lambda x: x.reshape(3, 3)[2, 1] - 0.3 * np.sum(x.reshape(3, 3)[2, :])},  # Blend 3 coffee variety 2 lower proportion constraint
    {'type': 'ineq', 'fun': lambda x: x.reshape(3, 3)[1, 1] + x.reshape(3, 3)[1, 2] - 0.7 * np.sum(x.reshape(3, 3)[1, :])}  # Blend 2 coffee varieties 2 and 3 constraint
]

# Define bounds (9 variables since we have a 3x3 array)
bounds = [(0, None)] * 9

# Initial guess (assume starting with 0 for all quantities)
initial_guess = np.zeros(9)

# Perform the optimization
result = minimize(profit, initial_guess, method='SLSQP', constraints=constraints, bounds=bounds)

# Reshape the result to a 2D array
Q_opt = result.x.reshape(3, 3)

# Print the results
print(f"Maximum profit is Rs. {-result.fun:.3f}")
print(f"Blend 1 has {Q_opt[0, 0]:.3f} Kgs, {Q_opt[0, 1]:.3f} Kgs, and {Q_opt[0, 2]:.3f} Kgs of Variety 1, 2, and 3 of coffee beans, respectively.")
print(f"Blend 2 has {Q_opt[1, 0]:.3f} Kgs, {Q_opt[1, 1]:.3f} Kgs, and {Q_opt[1, 2]:.3f} Kgs of Variety 1, 2, and 3 of coffee beans, respectively.")
print(f"Blend 3 has {Q_opt[2, 0]:.3f} Kgs, {Q_opt[2, 1]:.3f} Kgs, and {Q_opt[2, 2]:.3f} Kgs of Variety 1, 2, and 3 of coffee beans, respectively.")

Maximum profit is Rs. 4990000.000
Blend 1 has 1000.000 Kgs, 2000.000 Kgs, and 2000.000 Kgs of Variety 1, 2, and 3 of coffee beans, respectively.
Blend 2 has 6000.000 Kgs, 7000.000 Kgs, and 7000.000 Kgs of Variety 1, 2, and 3 of coffee beans, respectively.
Blend 3 has 0.000 Kgs, 0.000 Kgs, and 0.000 Kgs of Variety 1, 2, and 3 of coffee beans, respectively.


### Question 9

In [150]:
# Given data
yi = np.array([400, 600, 200])
pi = np.array([20, 15, 25])
fi = np.array([200, 300, 100])
ti = np.array([10, 12, 8])

# Objective coefficients (for minimization, negative of profit)
c = -(yi * pi) + (10 * fi + 40 * ti)

# Constraints
A = [
    [1, 1, 1],  # Total area constraint
    ti          # Total labor time constraint
]
b = [20, 2000]  # RHS of the constraints

# Bounds for each variable (non-negative)
x_bounds = [(0, None), (0, None), (0, None)]

# Perform linear programming optimization
result = linprog(c, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')

# Extract results
max_profit = -result.fun
area_allocation = result.x

# Print the results
print(f"Maximum profit is Rs. {max_profit:.3f}")
print(f"Product 1 is sown in an area of {area_allocation[0]:.3f} m^2.")
print(f"Product 2 is sown in an area of {area_allocation[1]:.3f} m^2.")
print(f"Product 3 is sown in an area of {area_allocation[2]:.3f} m^2.")

Maximum profit is Rs. 112000.000
Product 1 is sown in an area of 20.000 m^2.
Product 2 is sown in an area of 0.000 m^2.
Product 3 is sown in an area of 0.000 m^2.


### Question 10

In [151]:
l = 297
w = 210

def volume(x):
    return -(l-2*x)*(w-2*x)*x

bound = [(0,w/2)]

x0 = 1 #Intial Guess

result = minimize(volume, x0, method='SLSQP', bounds=bound)
print(f"The maximum volume of the open box is obtained as {-result.fun*10e-9:.3f} m^3 for the value of x as {result.x[0]:.3f} mm")

The maximum volume of the open box is obtained as 0.011 m^3 for the value of x as 40.423 mm
